In [5]:
import sys
import json
import numpy as np
import pandas as pd
import re

In [262]:
# Load the locations of a set of sensors and the closest beacon to each sensor
# For the input file, the row number is the sensor index, and the other data follows this pattern:
# "Sensor at x=<SENSOR X POSITION>, y=<SENSOR Y POSITION>: closest beacon is at x=<CLOSEST BEACON X POSITION>, y=<CLOSEST BEACON Y POSITION>"
# Load the sensors from the file "day15-inputsample.txt", storing as the tuple (sensor, beacon, distance)

class Sensor:
    def __init__(self, sensor, beacon):
        self.sensor = sensor
        self.beacon = beacon

        # distance between sensor and beacon
        distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
        self.distance = distance
        self.min_x = sensor[0] - distance
        self.max_x = sensor[0] + distance
        self.min_y = sensor[1] - distance
        self.max_y = sensor[1] + distance

    def __str__(self):
        return f"Sensor at x={self.sensor[0]}, y={self.sensor[1]}: closest beacon is at x={self.beacon[0]}, y={self.beacon[1]} distance={self.distance}, min/min x/y={self.min_x}/{self.min_y}, max/max x/y={self.max_x}/{self.max_y}"

def load_sensors(filename):
    sensors = []
    with open(filename) as f:
        for line in f:
            line = line.strip()
            # load the line using regex matching
            m = re.match(r'Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)', line)
            if m:
                sensors.append(Sensor((int(m.group(1)), int(m.group(2))), (int(m.group(3)), int(m.group(4)))))
            else:
                print(f'Error: {line}')
    return sensors

sample = load_sensors('day15-inputsample.txt')
print(len(sample))
[str(x) for x in sample]

14


['Sensor at x=2, y=18: closest beacon is at x=-2, y=15 distance=7, min/min x/y=-5/11, max/max x/y=9/25',
 'Sensor at x=9, y=16: closest beacon is at x=10, y=16 distance=1, min/min x/y=8/15, max/max x/y=10/17',
 'Sensor at x=13, y=2: closest beacon is at x=15, y=3 distance=3, min/min x/y=10/-1, max/max x/y=16/5',
 'Sensor at x=12, y=14: closest beacon is at x=10, y=16 distance=4, min/min x/y=8/10, max/max x/y=16/18',
 'Sensor at x=10, y=20: closest beacon is at x=10, y=16 distance=4, min/min x/y=6/16, max/max x/y=14/24',
 'Sensor at x=14, y=17: closest beacon is at x=10, y=16 distance=5, min/min x/y=9/12, max/max x/y=19/22',
 'Sensor at x=8, y=7: closest beacon is at x=2, y=10 distance=9, min/min x/y=-1/-2, max/max x/y=17/16',
 'Sensor at x=2, y=0: closest beacon is at x=2, y=10 distance=10, min/min x/y=-8/-10, max/max x/y=12/10',
 'Sensor at x=0, y=11: closest beacon is at x=2, y=10 distance=3, min/min x/y=-3/8, max/max x/y=3/14',
 'Sensor at x=20, y=14: closest beacon is at x=25, y=17

In [266]:
# create a dataframe from the sensors and beacons, marking sensors with an 'S', beacons with a 'B', and all other cells with NaN
def create_dataframe(sensors : list[Sensor]):
    # sensors are a list of tuples, with the first element being the sensor position, the second element being the beacon position, and the third element being the distance
    # find the overall max x and y values, using the distance outward from the sensor position in all directions
    max_x = max([s.max_x for s in sensors])
    max_y = max([s.max_y for s in sensors])
    min_x = min([s.min_x for s in sensors])
    min_y = min([s.min_y for s in sensors])
    
    # create a dataframe with the appropriate size
    cat_type = pd.api.types.CategoricalDtype(categories=["B","S","#"])

    # np.full((max_y - min_y + 1, max_x - min_x + 1), np.nan)
    df = pd.DataFrame(index=range(min_y, max_y + 1), columns=range(min_x, max_x + 1), dtype=cat_type)
    # add the sensors and beacons to the dataframe
    for s in sensors:
        df.loc[s.sensor[1], s.sensor[0]] = 'S'
        df.loc[s.beacon[1], s.beacon[0]] = 'B'
    return df

sample_df = create_dataframe(sample)
sample_df.to_clipboard()
sample_df

,-8,-7,-6,-5,-4,-3,-2,-1,0,1,...,19,20,21,22,23,24,25,26,27,28
-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
# find the distance between a sensor and its closest beacon, and draw a diamond out from the sensor of that size
def draw_diamond(df, sensor, distance):
    # draw a diamond outwards from the sensor position, centered on the sensor and extending out to the distance
    # draw a # for each cell in the diamond, only if it is currently empty
    for i in range(distance + 1):
        # draw the top and bottom lines
        for j in range(distance - i + 1):
            if df.loc[sensor[1] - i, sensor[0] + j] is np.nan:
                df.loc[sensor[1] - i, sensor[0] + j] = '#'
            if df.loc[sensor[1] + i, sensor[0] + j] is np.nan:
                df.loc[sensor[1] + i, sensor[0] + j] = '#'
        # draw the left and right lines
        for j in range(distance - i + 1):
            if df.loc[sensor[1] - i, sensor[0] - j] is np.nan:
                df.loc[sensor[1] - i, sensor[0] - j] = '#'
            if df.loc[sensor[1] + i, sensor[0] - j] is np.nan:
                df.loc[sensor[1] + i, sensor[0] - j] = '#'
        
    
    return df


In [278]:

diamond_sample = sample_df.copy()
for s in sample:
    diamond_sample = draw_diamond(diamond_sample, s.sensor, s.distance)

# diamond_sample = draw_diamond(diamond_sample, sample[2].sensor, sample[2].distance)

diamond_sample.to_clipboard()


In [271]:

# at row Y, how many cells are marked with a "#" or "S"
def count_unmarked(df, y):
    return len(df.loc[y, df.loc[y] == '#']) + len(df.loc[y, df.loc[y] == 'S'])

print(count_unmarked(diamond_sample, 10))

0


In [305]:
input = load_sensors('day15-input.txt')

# # input is a list of tuples, with the following elements:
# # 1. sensor position
# # 2. beacon position
# # 3. manhattan distance outwards from sensor position (diamond shape)
# # 4. min X of sensor range
# # 5. max X of sensor range
# # 6. min Y of sensor range
# # 7. max Y of sensor range

# # find the min and max X values and calculate the range
min_x = min([s.min_x for s in input])
max_x = max([s.max_x for s in input])
min_y = min([s.min_y for s in input])
max_y = max([s.max_y for s in input])
# print(min_x, max_x, max_x - min_x)

# construct a 1d array of None's between min_x and max_x
df = pd.DataFrame(index=range(min_x, max_x + 1), columns=[0], dtype=bool, data=True)


# for a given value of Y
# 1. Test to see if the sensor is in range of that Y value. If it is:
# 2. Distance to row = The number of rows between the sensor and that Y value
# 2. Leading spaces for row = Distance to row
# 3. Star count for row = (sensor count * 2 + 1) - distance to row
# 4. Return the range of the star count (start:end)
def get_star_range_for_row(y, sensor):
    if y >= sensor.min_y and y <= sensor.max_y:
        distance_to_row = abs(y - sensor.sensor[1])
        leading_spaces = distance_to_row
        star_count = ((sensor.distance * 2) + 1) - (leading_spaces * 2)
        # print("sensor at ", sensor[0], "distance to row ", distance_to_row, "min_x", sensor[3], "leading spaces ", leading_spaces, "star count ", star_count, "range ", leading_spaces, ":", leading_spaces + star_count - 1, "")
        return (leading_spaces, leading_spaces + star_count - 1)
    else:
        return None

def get_raw_ranges_for_row(y):
    ranges = []
    for s in input:
        r = get_star_range_for_row(y, s)
        if r is not None:
            # shift r by the sensor's min X value
            # print("sensor", s.sensor, "distance", s.distance, "min_x", s.min_x, "max_x", s.max_x, "min_y", s.min_y, "max_y", s.max_y)
            # print("return", r)
            r = (r[0] + s.min_x, r[1] + s.min_x)
            # print("shifted", r)
            ranges.append(r)
    return ranges


def merge_ranges(ranges):
    # merge overlapping ranges
    ranges.sort()
    merged_ranges = []
    for r in ranges:
        if len(merged_ranges) == 0:
            merged_ranges.append(r)
        else:
            if r[0] <= merged_ranges[-1][1] + 1:
                merged_ranges[-1] = (merged_ranges[-1][0], max(merged_ranges[-1][1], r[1]))
            else:
                merged_ranges.append(r)
    return merged_ranges

# print(get_ranges_for_row(11))

def get_range_size_for_row(y):
    ranges = merge_ranges(get_raw_ranges_for_row(y))
    size = 0
    for r in ranges:
        size += r[1] - r[0] + 1
    return size

# print(get_range_size_for_row(11))

def get_items_on_row(y):
    items = set()
    for s in input:
        if y == s.sensor[1]:
            # print('sensor', s.sensor)
            items.add(s.sensor)
        if y == s.beacon[1]:
            # print('beacon', s.beacon)
            items.add(s.beacon)
    return items

# for r in get_raw_ranges_for_row(10):
for r in get_raw_ranges_for_row(2000000):
    start, stop = r
    df.loc[start:stop] = False

# for item in get_items_on_row(10):
for item in get_items_on_row(2000000):
    df.loc[item[0]] = True

df[df == False].count()

0    5083287
dtype: int64

In [320]:
print(min_x, max_x, min_y, max_y)
def bound(i): return min(4000000, max(0, i))

for y in range(bound(min_y), bound(max_y + 1)):
    y_ranges = merge_ranges(get_raw_ranges_for_row(y))
    if len(y_ranges) == 2:
        # If there's only one spot available, then this will be the only gap
        x = y_ranges[0][1] + 1
        print("y", y, "ranges", y_ranges, "x", x)
        break

-1103113 5610424 -1470077 5556158
y 3205729 ranges [(-585465, 3283508), (3283510, 4284240)] x 3283509


In [322]:
x * 4000000 + y

13134039205729